In [9]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import CrossEntropyLoss
import time

# 데이터 로드 및 전처리
data_A = pd.read_csv("filtered_patients.csv")
data_B = pd.read_csv("filtered_conditions.csv")
data_merged = pd.merge(data_A, data_B, on="ID", how="inner")

selected_columns = ['BIRTHDATE', 'GENDER', 'RACE', 'ETHNICITY', 'DESCRIPTION']
data_selected = data_merged[selected_columns]

# COVID-19 감염 여부 판단
data_selected['INFECTED'] = data_selected['DESCRIPTION'].apply(lambda x: 1 if 'COVID-19' in str(x) else 0)

# 학습 데이터와 테스트 데이터로 나누기
X = data_selected[['BIRTHDATE', 'GENDER', 'RACE', 'ETHNICITY']]
y = data_selected['INFECTED']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# BERT 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(data):
    texts = [' '.join(map(str, [d])) for d in data]
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    return inputs

# 데이터셋 토큰화
train_inputs = tokenize_data(X_train)
test_inputs = tokenize_data(X_test)

# 레이블 데이터 변환
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Tensor 생성
train_labels = torch.tensor(y_train, dtype=torch.long)
test_labels = torch.tensor(y_test, dtype=torch.long)

# 데이터셋 크기 확인
if len(train_inputs['input_ids']) != len(train_inputs['attention_mask']) or len(train_inputs['input_ids']) != len(train_labels):
    raise ValueError("Size mismatch between tensors")

# DataLoader 생성
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_labels)

# 적절한 batch size 선택
train_batch_size = 64
test_batch_size = 64

# DataLoader 생성
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

class CustomBertForSequenceClassification(BertForSequenceClassification):
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
        output_hidden_states=True
    ):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            labels=labels,
            output_hidden_states=output_hidden_states
        )
        logits = outputs.logits
        hidden_states = outputs.hidden_states[-1]
        loss = outputs.loss
        return logits, loss, hidden_states

model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = CrossEntropyLoss()

# BERT 모델 학습
start_time = time.time()
model.train()
for epoch in range(3):
    total_loss = 0
    for batch_idx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        logits, loss, _ = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Batch {batch_idx+1}/{len(train_dataloader)}, Loss: {loss.item()}")  # 각 배치의 손실값 출력
    average_loss = total_loss / (batch_idx + 1)  # 배치의 수로 나눠줌
    print(f"Epoch {epoch+1} Average Loss: {average_loss}")

end_time = time.time()
print("Training complete.")
print(f"Training time: {end_time - start_time} seconds")

# BERT 모델 평가
model.eval()
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        logits, _, _ = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += torch.sum(predictions == labels).item()
        total_predictions += len(labels)
accuracy = correct_predictions / total_predictions
print(f"BERT Model Accuracy: {accuracy}")


<ipython-input-9-e5d018bbe259>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['INFECTED'] = data_selected['DESCRIPTION'].apply(lambda x: 1 if 'COVID-19' in str(x) else 0)


ValueError: Size mismatch between tensors

In [10]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import CrossEntropyLoss
from sklearn.ensemble import RandomForestClassifier
import time


# Step 1: 데이터 로드 및 전처리
data_A = pd.read_csv("filtered_patients.csv")
data_B = pd.read_csv("filtered_conditions.csv")
data_merged = pd.merge(data_A, data_B, on="ID", how="inner")

# print(data_merged)

selected_columns = ['ID', 'BIRTHDATE', 'GENDER', 'ADDRESS', 'DESCRIPTION']
def classify_covid19(description):
    if "COVID-19" in description:
        return 1
    elif "Suspected COVID-19" in description:
        return 1
    else:
        return 0

data_processed = data_merged[selected_columns].copy()
data_processed['GENDER'] = LabelEncoder().fit_transform(data_processed['GENDER'])
data_processed['COVID-19'] = data_processed['DESCRIPTION'].apply(classify_covid19)

# Step 2: 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data_processed['DESCRIPTION'], data_processed['COVID-19'], test_size=0.2, random_state=42)

# 테스트 데이터셋의 라벨 분포 확인
# print(y_test.value_counts())

# Step 3: BERT 모델 학습 및 평가
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(data):
    texts = [' '.join(map(str, [d])) for d in data]
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    return inputs

# 데이터셋 토큰화
train_inputs = tokenize_data(X_train)
test_inputs = tokenize_data(X_test)

# 레이블 데이터 변환
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Tensor 생성
train_labels = torch.tensor(y_train, dtype=torch.long)
test_labels = torch.tensor(y_test, dtype=torch.long)

# DataLoader 생성
train_batch_size = 64
test_batch_size = 64

train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_labels)
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

class CustomBertForSequenceClassification(BertForSequenceClassification):
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
        output_hidden_states=True
    ):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            labels=labels,
            output_hidden_states=output_hidden_states
        )
        logits = outputs.logits
        hidden_states = outputs.hidden_states[-1]
        loss = outputs.loss
        return logits, loss, hidden_states

# 모델 생성 또는 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = 'bert_model.pth'
save_hidden_states_path = 'hidden_states.npy'  # 추가된 부분: hidden states를 저장할 경로

if os.path.exists(model_path):
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.load_state_dict(torch.load(model_path))
    model.to(device)  # 모델을 GPU 또는 CPU로 이동
    print("Model loaded.")
else:
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(torch.cuda.is_available())
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    criterion = CrossEntropyLoss()

    # BERT 모델 학습
    start_time = time.time()
    model.train()
    for epoch in range(3):
        total_loss = 0
        for batch_idx, batch in enumerate(train_dataloader):
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            logits, loss, _ = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}/{len(train_dataloader)}, Loss: {loss.item()}")  # 각 배치의 손실값 출력
        average_loss = total_loss / (batch_idx + 1)  # 배치의 수로 나눠줌
        print(f"Epoch {epoch+1} Average Loss: {average_loss}")

    end_time = time.time()
    print("Training complete.")
    print(f"Training time: {end_time - start_time} seconds")

    # 모델 저장
    torch.save(model.state_dict(), model_path)
    
     # 추가된 부분: hidden states 저장
    np.save(save_hidden_states_path, train_features)  # 훈련 데이터셋의 hidden states를 저장

    # BERT 모델 평가
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            logits, _, _ = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(logits, dim=1)
            correct_predictions += torch.sum(predictions == labels).item()
            total_predictions += len(labels)
    accuracy = correct_predictions / total_predictions
    print(f"BERT Model Accuracy: {accuracy}")



# Step 4: BERT 모델의 hidden states를 활용하여 새로운 모델 학습 및 평가
train_features, train_labels = [], []
test_features, test_labels = [], []

def extract_features(data_loader, model):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_mask, label = batch
            input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
            _, _, hidden_states = model(input_ids=input_ids, attention_mask=attention_mask)
            features.append(hidden_states.cpu().numpy())
            labels.append(label.cpu().numpy())
    features = np.concatenate(features)
    labels = np.concatenate(labels)
    return features, labels

# BERT 모델의 hidden states를 활용하여 특징 추출
train_features, train_labels = extract_features(train_dataloader, model)
test_features, test_labels = extract_features(test_dataloader, model)

# Random Forest Classifier에 입력으로 사용할 수 있도록 특성 배열을 2D로 변환
train_features_2d = train_features.reshape(train_features.shape[0], -1)
test_features_2d = test_features.reshape(test_features.shape[0], -1)

# 추가된 부분: hidden states를 불러와서 새 모델 학습에 사용
if os.path.exists(save_hidden_states_path):
    train_features = np.load(save_hidden_states_path)
    print("Hidden states loaded.")
    # 새로운 모델(예: RandomForestClassifier) 학습
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(train_features_2d, train_labels)
else:
    print("Hidden states file not found. Run the BERT model to generate hidden states.")
    
# 새로운 모델 평가
accuracy = rf_model.score(test_features_2d, test_labels)
print(f"New Model Accuracy: {accuracy}")

# BERT 모델의 hidden states를 활용하여 복구한 데이터 출력
def recover_data_from_hidden_states(hidden_states, tokenizer):
    recovered_data = []
    for batch_hidden_states in hidden_states:
        for example_hidden_states in batch_hidden_states:
            # 각 토큰의 hidden state 중 가장 높은 값을 선택하여 복원
            max_hidden_state = np.max(example_hidden_states[1:], axis=0)  # 첫 번째 토큰([CLS])은 제외
            token_id = np.argmax(max_hidden_state)
            token_ids_list = [token_id]  # 리스트에 담아 반복 가능한 객체로 만듦
            recovered_text = tokenizer.decode(token_ids_list)
            recovered_data.append(recovered_text)
    return recovered_data

# BERT 모델의 hidden states를 활용하여 복구한 데이터 출력
print("Recovered Data from Hidden States:")
train_recovered_data = recover_data_from_hidden_states(train_features, tokenizer)
test_recovered_data = recover_data_from_hidden_states(test_features, tokenizer)

print("Train Recovered Data:")
print(train_recovered_data[:1000])  # 처음 100개의 복구된 데이터 출력
print("Test Recovered Data:")
print(test_recovered_data[:1000])  # 처음 100개의 복구된 데이터 출력

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded.
Hidden states loaded.
New Model Accuracy: 1.0
Recovered Data from Hidden States:
Train Recovered Data:
['[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [12]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.nn import CrossEntropyLoss
import torch.optim as optim
import torch.nn as nn
from sklearn.ensemble import RandomForestClassifier
import time


# Step 1: 데이터 로드 및 전처리
data_A = pd.read_csv("filtered_patients.csv")
data_B = pd.read_csv("filtered_conditions.csv")
data_merged = pd.merge(data_A, data_B, on="ID", how="inner")

# print(data_merged)

selected_columns = ['ID', 'BIRTHDATE', 'GENDER', 'ADDRESS', 'DESCRIPTION']
def classify_covid19(description):
    if "COVID-19" in description:
        return 1
    elif "Suspected COVID-19" in description:
        return 1
    else:
        return 0

data_processed = data_merged[selected_columns].copy()
data_processed['GENDER'] = LabelEncoder().fit_transform(data_processed['GENDER'])
data_processed['COVID-19'] = data_processed['DESCRIPTION'].apply(classify_covid19)

# Step 2: 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data_processed['DESCRIPTION'], data_processed['COVID-19'], test_size=0.2, random_state=42)

print("X_train\n", X_train)
print("y_train\n", y_train)

# 테스트 데이터셋의 라벨 분포 확인
# print(y_test.value_counts())

# Step 3: BERT 모델 학습 및 평가
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(data):
    texts = [' '.join(map(str, [d])) for d in data]
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    return inputs

# 데이터셋 토큰화
train_inputs = tokenize_data(X_train)
test_inputs = tokenize_data(X_test)

# 레이블 데이터 변환
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Tensor 생성
train_labels = torch.tensor(y_train, dtype=torch.long)
test_labels = torch.tensor(y_test, dtype=torch.long)

# DataLoader 생성
train_batch_size = 64
test_batch_size = 64

train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_labels)
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

class CustomBertForSequenceClassification(BertForSequenceClassification):
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
        output_hidden_states=True
    ):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            labels=labels,
            output_hidden_states=output_hidden_states
        )
        logits = outputs.logits
        hidden_states = outputs.hidden_states[-1]
        loss = outputs.loss
        return logits, loss, hidden_states

# 모델 생성 또는 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = 'bert_model.pth'
save_hidden_states_path = 'hidden_states.npy'  # 추가된 부분: hidden states를 저장할 경로

if os.path.exists(model_path):
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.load_state_dict(torch.load(model_path))
    model.to(device)  # 모델을 GPU 또는 CPU로 이동
    print("Model loaded.")
else:
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(torch.cuda.is_available())
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    criterion = CrossEntropyLoss()

    # BERT 모델 학습
    start_time = time.time()
    model.train()
    for epoch in range(3):
        total_loss = 0
        for batch_idx, batch in enumerate(train_dataloader):
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            logits, loss, _ = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}/{len(train_dataloader)}, Loss: {loss.item()}")  # 각 배치의 손실값 출력
        average_loss = total_loss / (batch_idx + 1)  # 배치의 수로 나눠줌
        print(f"Epoch {epoch+1} Average Loss: {average_loss}")

    end_time = time.time()
    print("Training complete.")
    print(f"Training time: {end_time - start_time} seconds")

    # 모델 저장
    torch.save(model.state_dict(), model_path)
    
     # 추가된 부분: hidden states 저장
    np.save(save_hidden_states_path, train_features)  # 훈련 데이터셋의 hidden states를 저장

    # BERT 모델 평가
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            logits, _, _ = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(logits, dim=1)
            correct_predictions += torch.sum(predictions == labels).item()
            total_predictions += len(labels)
    accuracy = correct_predictions / total_predictions
    print(f"BERT Model Accuracy: {accuracy}")



# 새로운 모델 클래스 정의
class CustomRecoveryModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CustomRecoveryModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# 데이터 준비
train_dataset = CustomDataset(train_features_2d, train_labels)
test_dataset = CustomDataset(test_features_2d, test_labels)

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델, 손실 함수, 옵티마이저 정의
model = CustomRecoveryModel(input_size=train_features_2d.shape[1], hidden_size=64, output_size=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

# 테스트 데이터로 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = torch.round(torch.sigmoid(outputs))
        total += labels.size(0)
        correct += (predicted == labels.view(-1, 1)).sum().item()

accuracy = correct / total
print(f"Accuracy: {accuracy}")

# 복원된 값을 출력하는 코드 추가
recovered_data = []

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        recovered_data_batch = torch.round(torch.sigmoid(outputs))
        recovered_data.extend(recovered_data_batch.cpu().numpy().flatten().tolist())

print("Recovered Data:")
print(recovered_data[:1000])  # 처음 10개의 복원된 데이터 출력


X_train
 54696                 Loss of taste (finding)
41913                       First degree burn
74118                 Loss of taste (finding)
57549          Respiratory distress (finding)
89528                      Suspected COVID-19
                         ...                 
76820                         Chill (finding)
110268    Acute pulmonary embolism (disorder)
103694                      Fatigue (finding)
860                        Suspected COVID-19
15795                                COVID-19
Name: DESCRIPTION, Length: 91635, dtype: object
y_train
 54696     0
41913     0
74118     0
57549     0
89528     1
         ..
76820     0
110268    0
103694    0
860       1
15795     1
Name: COVID-19, Length: 91635, dtype: int64


Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


True
Epoch 1, Batch 1/1432, Loss: 0.9207198619842529
Epoch 1, Batch 2/1432, Loss: 0.8558141589164734
Epoch 1, Batch 3/1432, Loss: 0.7800703048706055
Epoch 1, Batch 4/1432, Loss: 0.7023741602897644
Epoch 1, Batch 5/1432, Loss: 0.6612610816955566
Epoch 1, Batch 6/1432, Loss: 0.5973271131515503
Epoch 1, Batch 7/1432, Loss: 0.575785219669342
Epoch 1, Batch 8/1432, Loss: 0.588496744632721
Epoch 1, Batch 9/1432, Loss: 0.5424282550811768
Epoch 1, Batch 10/1432, Loss: 0.5458652973175049
Epoch 1, Batch 11/1432, Loss: 0.5472856163978577
Epoch 1, Batch 12/1432, Loss: 0.513495922088623
Epoch 1, Batch 13/1432, Loss: 0.5053238272666931
Epoch 1, Batch 14/1432, Loss: 0.4773605465888977
Epoch 1, Batch 15/1432, Loss: 0.47765907645225525
Epoch 1, Batch 16/1432, Loss: 0.4523170590400696
Epoch 1, Batch 17/1432, Loss: 0.44228464365005493
Epoch 1, Batch 18/1432, Loss: 0.4149090647697449
Epoch 1, Batch 19/1432, Loss: 0.3964105248451233
Epoch 1, Batch 20/1432, Loss: 0.3746567666530609
Epoch 1, Batch 21/1432, L

Epoch 1, Batch 163/1432, Loss: 0.0036072521470487118
Epoch 1, Batch 164/1432, Loss: 0.0033789852168411016
Epoch 1, Batch 165/1432, Loss: 0.0033729197457432747
Epoch 1, Batch 166/1432, Loss: 0.0032653675880283117
Epoch 1, Batch 167/1432, Loss: 0.0031512302812188864
Epoch 1, Batch 168/1432, Loss: 0.003301331540569663
Epoch 1, Batch 169/1432, Loss: 0.003323799464851618
Epoch 1, Batch 170/1432, Loss: 0.003357558511197567
Epoch 1, Batch 171/1432, Loss: 0.0036757327616214752
Epoch 1, Batch 172/1432, Loss: 0.003202756866812706
Epoch 1, Batch 173/1432, Loss: 0.002844445873051882
Epoch 1, Batch 174/1432, Loss: 0.00297245429828763
Epoch 1, Batch 175/1432, Loss: 0.0029794189613312483
Epoch 1, Batch 176/1432, Loss: 0.003118538996204734
Epoch 1, Batch 177/1432, Loss: 0.0028570308350026608
Epoch 1, Batch 178/1432, Loss: 0.0031137133482843637
Epoch 1, Batch 179/1432, Loss: 0.003085720119997859
Epoch 1, Batch 180/1432, Loss: 0.00271443254314363
Epoch 1, Batch 181/1432, Loss: 0.0027741033118218184
Epoc

Epoch 1, Batch 319/1432, Loss: 0.0012561645125970244
Epoch 1, Batch 320/1432, Loss: 0.0013107002014294267
Epoch 1, Batch 321/1432, Loss: 0.0012233529705554247
Epoch 1, Batch 322/1432, Loss: 0.0012690563453361392
Epoch 1, Batch 323/1432, Loss: 0.0012984576169401407
Epoch 1, Batch 324/1432, Loss: 0.00123692792840302
Epoch 1, Batch 325/1432, Loss: 0.001317961374297738
Epoch 1, Batch 326/1432, Loss: 0.0012753861956298351
Epoch 1, Batch 327/1432, Loss: 0.0012396989623084664
Epoch 1, Batch 328/1432, Loss: 0.0011902584228664637
Epoch 1, Batch 329/1432, Loss: 0.0012286477722227573
Epoch 1, Batch 330/1432, Loss: 0.0010906162206083536
Epoch 1, Batch 331/1432, Loss: 0.0011960135307163
Epoch 1, Batch 332/1432, Loss: 0.001207471708767116
Epoch 1, Batch 333/1432, Loss: 0.0011885161511600018
Epoch 1, Batch 334/1432, Loss: 0.0012086504139006138
Epoch 1, Batch 335/1432, Loss: 0.0011938484385609627
Epoch 1, Batch 336/1432, Loss: 0.0012079435400664806
Epoch 1, Batch 337/1432, Loss: 0.0011433230247348547


...

Epoch 3, Batch 843/1432, Loss: 2.0693762053269893e-05
Epoch 3, Batch 844/1432, Loss: 2.0019493604195304e-05
Epoch 3, Batch 845/1432, Loss: 2.0185267203487456e-05
Epoch 3, Batch 846/1432, Loss: 2.0924720956827514e-05
Epoch 3, Batch 847/1432, Loss: 2.013125595112797e-05
Epoch 3, Batch 848/1432, Loss: 2.0639745343942195e-05
Epoch 3, Batch 849/1432, Loss: 2.088374094455503e-05
Epoch 3, Batch 850/1432, Loss: 2.0682589820353314e-05
Epoch 3, Batch 851/1432, Loss: 2.014615347434301e-05
Epoch 3, Batch 852/1432, Loss: 2.0045570636284538e-05
Epoch 3, Batch 853/1432, Loss: 2.0816689357161522e-05
Epoch 3, Batch 854/1432, Loss: 2.0146155293332413e-05
Epoch 3, Batch 855/1432, Loss: 2.1284204194671474e-05
Epoch 3, Batch 856/1432, Loss: 2.116312680300325e-05
Epoch 3, Batch 857/1432, Loss: 2.0149876945652068e-05
Epoch 3, Batch 858/1432, Loss: 2.1513300453079864e-05
Epoch 3, Batch 859/1432, Loss: 1.9572467863326892e-05
Epoch 3, Batch 860/1432, Loss: 2.134753049176652e-05
Epoch 3, Batch 861/1432, Loss: 2.

Epoch 3, Batch 997/1432, Loss: 1.844745384005364e-05
Epoch 3, Batch 998/1432, Loss: 1.934710053319577e-05
Epoch 3, Batch 999/1432, Loss: 2.0025074263685383e-05
Epoch 3, Batch 1000/1432, Loss: 1.9261417037341744e-05
Epoch 3, Batch 1001/1432, Loss: 1.9224164134357125e-05
Epoch 3, Batch 1002/1432, Loss: 1.8197864847024903e-05
Epoch 3, Batch 1003/1432, Loss: 1.9600407540565357e-05
Epoch 3, Batch 1004/1432, Loss: 1.8337563233217224e-05
Epoch 3, Batch 1005/1432, Loss: 1.8903792806668207e-05
Epoch 3, Batch 1006/1432, Loss: 1.8125227143173106e-05
Epoch 3, Batch 1007/1432, Loss: 1.9730778149096295e-05
Epoch 3, Batch 1008/1432, Loss: 1.9669325411086902e-05
Epoch 3, Batch 1009/1432, Loss: 1.8240711142425425e-05
Epoch 3, Batch 1010/1432, Loss: 1.907515434140805e-05
Epoch 3, Batch 1011/1432, Loss: 1.900437564472668e-05
Epoch 3, Batch 1012/1432, Loss: 1.8430697309668176e-05
Epoch 3, Batch 1013/1432, Loss: 1.882742981251795e-05
Epoch 3, Batch 1014/1432, Loss: 1.8060036381939426e-05
Epoch 3, Batch 101

Epoch 3, Batch 1149/1432, Loss: 1.7149217455880716e-05
Epoch 3, Batch 1150/1432, Loss: 1.6461912309750915e-05
Epoch 3, Batch 1151/1432, Loss: 1.7199508874909952e-05
Epoch 3, Batch 1152/1432, Loss: 1.671150494075846e-05
Epoch 3, Batch 1153/1432, Loss: 1.7946415027836338e-05
Epoch 3, Batch 1154/1432, Loss: 1.646563941903878e-05
Epoch 3, Batch 1155/1432, Loss: 1.6979718566290103e-05
Epoch 3, Batch 1156/1432, Loss: 1.831893496273551e-05
Epoch 3, Batch 1157/1432, Loss: 1.6681702618370764e-05
Epoch 3, Batch 1158/1432, Loss: 1.7486343494965695e-05
Epoch 3, Batch 1159/1432, Loss: 1.725352012726944e-05
Epoch 3, Batch 1160/1432, Loss: 1.73652806552127e-05
Epoch 3, Batch 1161/1432, Loss: 1.696854451438412e-05
Epoch 3, Batch 1162/1432, Loss: 1.6808358850539662e-05
Epoch 3, Batch 1163/1432, Loss: 1.7115688024205156e-05
Epoch 3, Batch 1164/1432, Loss: 1.7130589185399003e-05
Epoch 3, Batch 1165/1432, Loss: 1.6996484191622585e-05
Epoch 3, Batch 1166/1432, Loss: 1.5815587175893597e-05
Epoch 3, Batch 11

Epoch 3, Batch 1299/1432, Loss: 1.599440111021977e-05
Epoch 3, Batch 1300/1432, Loss: 1.6191832401091233e-05
Epoch 3, Batch 1301/1432, Loss: 1.469988092139829e-05
Epoch 3, Batch 1302/1432, Loss: 1.5325716958614066e-05
Epoch 3, Batch 1303/1432, Loss: 1.606704063306097e-05
Epoch 3, Batch 1304/1432, Loss: 1.5832349163247272e-05
Epoch 3, Batch 1305/1432, Loss: 1.5919891666271724e-05
Epoch 3, Batch 1306/1432, Loss: 1.57373578986153e-05
Epoch 3, Batch 1307/1432, Loss: 1.557903306093067e-05
Epoch 3, Batch 1308/1432, Loss: 1.5869600247242488e-05
Epoch 3, Batch 1309/1432, Loss: 1.566098944749683e-05
Epoch 3, Batch 1310/1432, Loss: 1.593479282746557e-05
Epoch 3, Batch 1311/1432, Loss: 1.538532342237886e-05
Epoch 3, Batch 1312/1432, Loss: 1.551384229969699e-05
Epoch 3, Batch 1313/1432, Loss: 1.6502890503033996e-05
Epoch 3, Batch 1314/1432, Loss: 1.6081941794254817e-05
Epoch 3, Batch 1315/1432, Loss: 1.6555039110244252e-05
Epoch 3, Batch 1316/1432, Loss: 1.5219548004097305e-05
Epoch 3, Batch 1317/

NameError: name 'train_features' is not defined